In [4]:
import pathlib
from argparse import ArgumentParser
import yaml
import json
import socket
import os

from importlib import reload
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from src.classifiers import *
from corpus.CIFAR100_dataset import *

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
import torch.nn.functional as F

import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet18, resnet34, resnet50
import torchvision.models.resnet as Resnet

import pytorch_lightning as pl

In [5]:
seed_everything(1)

[rank: 0] Seed set to 1


1

In [14]:
from torchsummary import summary

In [8]:
model = resnet18(Resnet.Bottleneck, [2,2,2,2], pretrained=False)

In [11]:
model.inplanes=64
model.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
model.bn1 = nn.BatchNorm2d(64)
model.fc = nn.Linear(512, 100)
del model.maxpool
model.maxpool = lambda x : x

In [31]:
layer_idx = 0
for data in model.named_parameters():
    name, param = data
    if param.requires_grad:
        if 'bn' in name or 'bias' in name or 'downsample' in name:
            continue
        else:
            print(name)
            layer_idx += 1
print(layer_idx)

conv1.weight
layer1.0.conv1.weight
layer1.0.conv2.weight
layer1.1.conv1.weight
layer1.1.conv2.weight
layer2.0.conv1.weight
layer2.0.conv2.weight
layer2.1.conv1.weight
layer2.1.conv2.weight
layer3.0.conv1.weight
layer3.0.conv2.weight
layer3.1.conv1.weight
layer3.1.conv2.weight
layer4.0.conv1.weight
layer4.0.conv2.weight
layer4.1.conv1.weight
layer4.1.conv2.weight
fc.weight
18


In [15]:
info = summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,864
       BatchNorm2d-8           [-1, 64, 32, 32]             128
              ReLU-9           [-1, 64, 32, 32]               0
       BasicBlock-10           [-1, 64, 32, 32]               0
           Conv2d-11           [-1, 64, 32, 32]          36,864
      BatchNorm2d-12           [-1, 64, 32, 32]             128
             ReLU-13           [-1, 64, 32, 32]               0
           Conv2d-14           [-1, 64,

In [ ]:
from torch.autograd import Variable
from collections import OrderedDict
import numpy as np

In [38]:
def layer_list(model, input_size, dtypes=None):
    device = model.device
    if dtypes == None:
        dtypes = [torch.FloatTensor]*len(input_size)

    def register_hook(module):
        def hook(module, input, output):
            try:
                weight_name = list(module.state_dict().keys())[0]
            except:
                weight_name = ''
            if 'conv' in weight_name or 'fc' in weight_name:
                print(module.__class__)
                if isinstance(output, (list, tuple)):
                    summary.extend([list(o.size())[1:] for o in output])
                else:
                    summary.append(list(output.size())[1:])
        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
        ):
            hooks.append(module.register_forward_hook(hook))

    # multiple inputs to the network
    if isinstance(input_size, tuple):
        input_size = [input_size]

    # batch_size of 2 for batchnorm
    x = [torch.rand(2, *in_size).type(dtype).to(device=device)
         for in_size, dtype in zip(input_size, dtypes)]

    # create properties
    summary = []
    hooks = []

    # register hook
    model.apply(register_hook)

    # make a forward pass
    model(*x)

    # remove these hooks
    for h in hooks:
        h.remove()
    summary = [size for size in summary[:-1] for _ in range(2)]
    summary = [[3, 32, 32], [64, 32, 32]] + summary + [[100]]
    layer_sizes = [np.prod(size) for size in summary]
    return layer_sizes

In [37]:
layer_list(model, (3, 32, 32))

TypeError: 'odict_keys' object is not subscriptable

In [ ]:
model.target_norms